In [ ]:
import numpy as np
from PIL import Image
from ImageFrame import Frame
from EdgeFinderV4 import BoundaryDetector as BD2
import timeit
import cv2 as cv
from numba import njit

In [ ]:
img = np.array(Image.open("TestingImage/Image3.jpg"))
img1 =Image.open("TestingImage/Image3.jpg")
frame1 = Frame(1280,720,10)

In [ ]:
kernal = np.array([-1,-1,0,1,1])
steps = 0
offset = (img[0].size-9)%(kernal.size+steps)
T_hold = 21

In [ ]:
for i in range(50):
    img = np.array(Image.open("TestingImage/Image"+str(i)+".jpg"))
    t1 = timeit.default_timer()
    B = BD2(img,kernal,frame1,steps,T_hold)
    t2 = timeit.default_timer()
    B.ploting_onImage(img,t2-t1)

In [ ]:
kernal = np.array([-1,-1,0,1,1])
steps = 0
offset = (img[0].size-9)%(kernal.size+steps)
T_hold = 21
points = np.empty([0,2])

t1 = timeit.default_timer()
for i,fline in enumerate(frame1.fline.values()):
    detection = False
    for t in range(20,fline.shape[0]-offset,kernal.size+steps):
        #for x,y in fline[t:t+kernal.size]:
         #   datas_segment = np.append(datas_segment,img[y,x,0])
        #dot_result = kernal.dot(datas_segment)
        #dot_result = kernal.dot([img[y,x,0] for x,y in fline[t:t+kernal.size]])
        #print(dot_result)
        if np.abs(kernal.dot([img[y,x,0] for x,y in fline[t:t+kernal.size]])) > T_hold:
            #print(t)
            points=np.append(points,[fline[t]],axis=0)
            #boundary_index += [t]
            detection = True
            break
    if not detection:
        points=np.append(points,[[0,0]],axis=0)
        #boundary_index += [0]
        detection = False
t2 = timeit.default_timer()
print(f'time is {t2-t1}')

In [ ]:
print(points)

In [ ]:
data = frame1.get_data(img1,0)

%timeit B =BD(data,kernal,frame1,steps,T_hold)


In [ ]:
frame1_values =np.array(list(frame1.fline.values()))

In [ ]:
def boundary_function(img,offset,kernal,frame1_values,steps,T_hold,sp):
    points = np.zeros_like(sp)
    for i,fline in enumerate(frame1_values):    
        detection = False
        for t in range(20,fline.shape[0]-offset,kernal.size+steps):
            if np.abs(np.sum(kernal*np.array([img[y,x,0] for x,y in fline[t:t+kernal.size]]))) > T_hold:
                #print(t)
                points[i]= fline[t]
                #boundary_index += [t]
                detection = True
                break
        if not detection:
            points = np.array([0,0])
            #boundary_index += [0]
            detection = False
    return points

In [ ]:
kernal = np.array([-1,-1,0,1,1],np.int64)

In [ ]:

b= boundary_function(img,offset,kernal,frame1_values,steps,T_hold,np.empty([8,2]))
print(b)

In [ ]:
functioned_jit = njit()(boundary_function)

In [ ]:
b2 = functioned_jit(img,offset,kernal,frame1_values,steps,T_hold,np.empty([8,2]))
print(b2)

In [ ]:
open_img = cv.imread("TestingImage/Image3.jpg")
%timeit dst = cv.Canny(open_img, 100, 200, None, 3)